In [1]:
import keras
import tensorflow as tf

import os
tf.config.optimizer.set_jit(False)  # Desativa XLA

2024-09-23 14:17:06.692173: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-23 14:17:06.706710: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-23 14:17:06.710840: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-23 14:17:06.721537: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-23 14:17:07.625728: W tensorflow/compiler/tf2

In [2]:
from segmentandoDatasets import *

segmentando_datasets(10000,10000,10000)

DataFrame do local PUC salvo como: Datasets_csv/df_PUC.csv
DataFrame do local PUC:
                                      caminho_imagem    classe
0  /home/lucas/Downloads/PKLot/PKLotSegmented/PUC...     Empty
1  /home/lucas/Downloads/PKLot/PKLotSegmented/PUC...  Occupied
2  /home/lucas/Downloads/PKLot/PKLotSegmented/PUC...  Occupied
3  /home/lucas/Downloads/PKLot/PKLotSegmented/PUC...     Empty
4  /home/lucas/Downloads/PKLot/PKLotSegmented/PUC...  Occupied


DataFrame do local UFPR04 salvo como: Datasets_csv/df_UFPR04.csv
DataFrame do local UFPR04:
                                      caminho_imagem    classe
0  /home/lucas/Downloads/PKLot/PKLotSegmented/UFP...     Empty
1  /home/lucas/Downloads/PKLot/PKLotSegmented/UFP...  Occupied
2  /home/lucas/Downloads/PKLot/PKLotSegmented/UFP...     Empty
3  /home/lucas/Downloads/PKLot/PKLotSegmented/UFP...     Empty
4  /home/lucas/Downloads/PKLot/PKLotSegmented/UFP...     Empty


DataFrame do local UFPR05 salvo como: Datasets_csv/df_UFPR05.csv


In [3]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from typing import List, Tuple
import os
import albumentations as A

from Preprocessamento import preprocessamento, preprocessamento_dataframe

/home/lucas/PIBIC (copy)/.venv/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
csv_file = 'Datasets_csv/df_PUC.csv'
treino_gerador, validacao_gerador, teste_gerador, treino, validacao, teste = preprocessamento(csv_file, autoencoder=False)

Found 6000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [5]:
from Modelos import GeradorAutoencoder

gerador = GeradorAutoencoder()
autoencoder = gerador.gerar_modelo()

autoencoder.summary()

encoder = autoencoder.get_layer('Encoder')
decoder = autoencoder.get_layer('Decoder')

/home/lucas/PIBIC (copy)/.venv/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
I0000 00:00:1727111830.335468   56646 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727111830.369351   56646 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1727111830.369522   56646 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at http

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 16, 16, 32)     │        40,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 64, 64, 3)      │        49,699 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 90,179 (352.26 KB)

 Trainable params: 90,179 (352.26 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
autoencoder.compile(loss="mse", optimizer='adam')

In [7]:
autoencoder.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Encoder (Sequential)            │ (None, 16, 16, 32)     │        40,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Decoder (Sequential)            │ (None, 64, 64, 3)      │        49,699 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 90,179 (352.26 KB)

 Trainable params: 90,179 (352.26 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Verifique a forma da saída do autoencoder
output_shape = autoencoder.output_shape
print(f"Forma de saída do autoencoder: {output_shape}")  # Deve ser (None, 64, 64, 3)


Forma de saída do autoencoder: (None, 64, 64, 3)


In [11]:
history = autoencoder.fit(
    treino_gerador,
    steps_per_epoch=6000 // 32,
    epochs=30,
    validation_data=validacao_gerador,
    validation_steps=2000 // 32
)


Epoch 1/30


2024-09-23 14:19:10.012230: W tensorflow/core/framework/op_kernel.cc:1840] OP_REQUIRES failed at xla_ops.cc:577 : INVALID_ARGUMENT: Incompatible shapes: [32] vs. [32,64,64,3]
	 [[{{node compile_loss/mse/sub}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_2158[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
2024-09-23 14:19:10.012268: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [32] vs. [32,64,64,3]
	 [[{{node compile_loss/mse/sub}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_2158[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]]


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/mse/sub defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [32] vs. [32,64,64,3]
	 [[{{node compile_loss/mse/sub}}]]
	tf2xla conversion failed while converting __inference_one_step_on_data_2158[]. Run with TF_DUMP_GRAPH_PREFIX=/path/to/dump/dir and --vmodule=xla_compiler=2 to obtain a dump of the compiled functions.
	 [[StatefulPartitionedCall]] [Op:__inference_one_step_on_iterator_2229]

In [10]:
pd.DataFrame(history.history).plot()
autoencoder.save("Modelos_keras/Autoencoder_teste.keras")
autoencoder.save_weights("weights_finais/Autoencoder_teste.weights.h5")

In [ ]:
autoencoder.load_weights("weights_finais/Autoencoder.weights.h5")

In [ ]:
print(len(treino_gerador))

In [13]:
import matplotlib.pyplot as plt
import numpy as np

def plot_autoencoder(x_test, Autoencoder):
    def normalize(image):
        image = np.clip(image, 0, 1)  
        return (image - image.min()) / (image.max() - image.min()) if image.max() != image.min() else image

    plt.figure(figsize=(16, 8))

    for i in range(8):
        # Imagem original
        plt.subplot(2, 8, i + 1)
        plt.imshow(x_test[i])
        plt.title("Original")
        plt.axis("off")

        # Predição e normalização
        pred = Autoencoder.predict(x_test[i].reshape((1, 64, 64, 3)))
        pred_img = normalize(pred[0])

        plt.subplot(2, 8, i + 8 + 1)
        plt.imshow(pred_img)
        plt.title("Reconstruída")
        plt.axis("off")

    plt.show()


In [14]:
x_treino, y_treino = next(treino_gerador)

In [ ]:
UFPR04, UFPR04_df = preprocessamento_dataframe('Datasets_csv/df_UFPR04.csv')
UFPR05, UFPR05_df = preprocessamento_dataframe('Datasets_csv/df_UFPR05.csv')

x_UFPR04, _ = next(UFPR04)
x_UFPR05, _ = next(UFPR05)

In [ ]:
plot_autoencoder(x_treino, autoencoder)
plot_autoencoder(x_UFPR04, autoencoder)
plot_autoencoder(x_UFPR05, autoencoder)

In [17]:
from keras import layers, models
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization

classifier = models.Sequential([
    encoder, 
    layers.Flatten(),  
    layers.Dropout(0.3),  
    layers.Dense(128, activation='relu'),  
    layers.Dense(2, activation='softmax')  
], name='classifier')

In [18]:
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
classifier.load_weights("/home/lucas/PIBIC (copy)/weights_finais/Autoencoder.weights.h5", skip_mismatch=True) #skip_mismatch para conseguir pegar os pesos do autoencoder

In [ ]:
#history = classifier.fit(treino_gerador, epochs=10, batch_size=16, validation_data=validacao_gerador)

In [22]:
#classifier.save("/home/lucas/PIBIC (copy)/Modelos_keras/classifier.keras")
#classifier.save_weights("/home/lucas/PIBIC (copy)/Modelos_keras/classifier.weights.h5")

In [23]:
#pd.DataFrame(history.history).plot()

In [ ]:
classifier.load_weights("/home/lucas/PIBIC (copy)/Modelos_keras/classifier.weights.h5")

In [ ]:
predicoes = classifier.predict(teste_gerador)
predicoes = np.argmax(predicoes, axis=1)

In [26]:
from preprocessamento import mapear_rotulos_binarios

y_verdadeiro = mapear_rotulos_binarios(teste['classe'])

In [27]:
from visualizacao import plot_confusion_matrix

plot_confusion_matrix(y_verdadeiro, predicoes, ['Empty', 'Occupied'], "/home/lucas/PIBIC (copy)/Resultados/Matriz_de_confusao/Autoencoder/Autoencoder_PUC", 'PUC')

In [ ]:
predicoes_UFPR04 = classifier.predict(UFPR04)
predicoes_UFPR05 = classifier.predict(UFPR05)

predicoes_UFPR04 = np.argmax(predicoes_UFPR04, axis=1)
predicoes_UFPR05 = np.argmax(predicoes_UFPR05, axis=1)

y_UFPR04 = mapear_rotulos_binarios(UFPR04_df['classe'])
y_UFPR05 = mapear_rotulos_binarios(UFPR05_df['classe'])

plot_confusion_matrix(y_UFPR04, predicoes_UFPR04, ['Empty', 'Occupied'], "/home/lucas/PIBIC (copy)/Resultados/Matriz_de_confusao/Autoencoder/Autoencoder_UFPR04", 'UFPR04')
plot_confusion_matrix(y_UFPR05, predicoes_UFPR05, ['Empty', 'Occupied'], "/home/lucas/PIBIC (copy)/Resultados/Matriz_de_confusao/Autoencoder/Autoencoder_UFPR05", 'UFPR05')